# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [1]:
from DataFuncs import normalize_train_data, train_test_split
normalize_train_data("combine")

In [2]:
train_test_split(0.2)

In [ ]:
from PIL import Image
img = Image.open("test.jpg").convert('L')
img = img.resize((5000,500))
img.show()